In [1]:
import pandas as pd
import numpy as np

# Random seed for reproducibility
seed = 202
np.random.seed(seed)

# Ignore warnings
import warnings
warnings.simplefilter('ignore')

# Import the train set and test set
train_data = pd.read_csv("train.csv", delimiter=",")
test_data = pd.read_csv("test.csv", delimiter=",")
train_data.shape, test_data.shape

((903653, 12), (804684, 12))

In [2]:
train_data.head()

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147
2,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,1472865386
3,Organic Search,20160902,"{""browser"": ""UC Browser"", ""browserVersion"": ""n...",4763447161404445595,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",4763447161404445595_1472881213,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472881213,1,1472881213
4,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",27294437909732085,"{""continent"": ""Europe"", ""subContinent"": ""North...",27294437909732085_1472822600,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472822600,2,1472822600


In [3]:
train_data.isnull().sum()

channelGrouping         0
date                    0
device                  0
fullVisitorId           0
geoNetwork              0
sessionId               0
socialEngagementType    0
totals                  0
trafficSource           0
visitId                 0
visitNumber             0
visitStartTime          0
dtype: int64

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 12 columns):
channelGrouping         903653 non-null object
date                    903653 non-null int64
device                  903653 non-null object
fullVisitorId           903653 non-null object
geoNetwork              903653 non-null object
sessionId               903653 non-null object
socialEngagementType    903653 non-null object
totals                  903653 non-null object
trafficSource           903653 non-null object
visitId                 903653 non-null int64
visitNumber             903653 non-null int64
visitStartTime          903653 non-null int64
dtypes: int64(4), object(8)
memory usage: 82.7+ MB


In [6]:
jsoncols = ['geoNetwork', 'device', 'totals', 'trafficSource']


def unpack(df):
    for jc in jsoncols:  # parse json
        flat_df = pd.DataFrame(df.pop(jc).apply(pd.io.json.loads).values.tolist())
        flat_df.columns = ['{}_{}'.format(jc, c) for c in flat_df.columns]
        df = df.join(flat_df)
    return df

In [7]:
train = unpack(train_data)
## drop sessionId, totals_visits, socialEngagementType

In [8]:
train.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,geoNetwork_city,geoNetwork_cityId,...,totals_visits,trafficSource_adContent,trafficSource_adwordsClickInfo,trafficSource_campaign,trafficSource_campaignCode,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Izmir,not available in demo dataset,...,1,NaN,{'criteriaParameters': 'not available in demo ...,(not set),NaN,NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,not available in demo dataset,not available in demo dataset,...,1,NaN,{'criteriaParameters': 'not available in demo ...,(not set),NaN,NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Madrid,not available in demo dataset,...,1,NaN,{'criteriaParameters': 'not available in demo ...,(not set),NaN,NaN,(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,not available in demo dataset,not available in demo dataset,...,1,NaN,{'criteriaParameters': 'not available in demo ...,(not set),NaN,NaN,google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,not available in demo dataset,not available in demo dataset,...,1,NaN,{'criteriaParameters': 'not available in demo ...,(not set),NaN,True,(not provided),organic,NaN,google


In [9]:
train.isnull().sum()

channelGrouping                          0
date                                     0
fullVisitorId                            0
sessionId                                0
socialEngagementType                     0
visitId                                  0
visitNumber                              0
visitStartTime                           0
geoNetwork_city                          0
geoNetwork_cityId                        0
geoNetwork_continent                     0
geoNetwork_country                       0
geoNetwork_latitude                      0
geoNetwork_longitude                     0
geoNetwork_metro                         0
geoNetwork_networkDomain                 0
geoNetwork_networkLocation               0
geoNetwork_region                        0
geoNetwork_subContinent                  0
device_browser                           0
device_browserSize                       0
device_browserVersion                    0
device_deviceCategory                    0
device_flas

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 50 columns):
channelGrouping                     903653 non-null object
date                                903653 non-null int64
fullVisitorId                       903653 non-null object
sessionId                           903653 non-null object
socialEngagementType                903653 non-null object
visitId                             903653 non-null int64
visitNumber                         903653 non-null int64
visitStartTime                      903653 non-null int64
geoNetwork_city                     903653 non-null object
geoNetwork_cityId                   903653 non-null object
geoNetwork_continent                903653 non-null object
geoNetwork_country                  903653 non-null object
geoNetwork_latitude                 903653 non-null object
geoNetwork_longitude                903653 non-null object
geoNetwork_metro                    903653 non-null object
geoNetwo

In [11]:
train['totals_transactionRevenue'][train['totals_transactionRevenue'].notnull()]

752        37860000
753       306670000
799        68030000
802        26250000
859       574150000
866         8380000
893       395730000
910        24080000
922        35480000
925        35080000
930        81500000
942       546320000
971        63840000
974       117340000
976        43710000
999        34480000
1000       35390000
1008      339030000
1020       33670000
1022      635500000
1026      103240000
1027      305470000
1037        5150000
1041       90850000
1054       10590000
1055       25680000
1056      338970000
2774      593910000
3161       44790000
3268       30390000
            ...    
899951     43430000
903089     44790000
903118    175800000
903122     87990000
903167     14370000
903168    155970000
903189     75180000
903194    350000000
903201    135920000
903211    159980000
903222     23180000
903228     22970000
903234    463600000
903235     79990000
903243     76780000
903249     12990000
903256     34970000
903259     91080000
903262     46110000


In [12]:
len(np.unique(train['totals_visits']))

1

In [32]:
import gc
import numpy as np
import pandas as pd

nan_list = ["not available in demo dataset",
            "unknown.unknown",
            "(not provided)",
            "(not set)"] 
nan_dict = {nl:np.nan for nl in nan_list}


def date_conv(df):
    # make a lookup table
    datevals = pd.date_range(start='2016-08-01', end='2018-04-30')
    datekeys = datevals.astype(str)
    datekeys = [d.replace('-', '') for d in datekeys]
    datedict = dict(zip(datekeys, datevals))
    # lookup
    df['date'] = df.date.map(datedict)
    return df


def unpack(df):
    for jc in jsoncols:  # parse json
        flat_df = pd.DataFrame(df.pop(jc).apply(pd.io.json.loads).values.tolist())
        flat_df.columns = ['{}_{}'.format(jc, c) for c in flat_df.columns]
        df = df.join(flat_df)
    ad_df = df.pop('trafficSource_adwordsClickInfo').apply(pd.Series) # handle dict column
    ad_df.columns = ['tS_adwordsCI_{}'.format(c) for c in ad_df.columns]
    df = df.join(ad_df)
    return df


def clean(df):
    df.drop(['sessionId', 'tS_adwordsCI_targetingCriteria', 'totals_visits', 'socialEngagementType'], 
        axis=1, inplace=True)
    df.replace(nan_dict, inplace=True) # convert disguised NaNs
    df.dropna(axis=1, how='all', inplace=True) 
    for col in df.columns:
        if 'totals' in col: # chnage to numeric
            df[col] = pd.to_numeric(df[col])
    df.totals_bounces.fillna(value=0, inplace=True)
    df.totals_newVisits.fillna(value=0, inplace=True)
    df.trafficSource_isTrueDirect.fillna(value=False, inplace=True)
    df.tS_adwordsCI_isVideoAd.fillna(value=True, inplace=True)
    return df

In [33]:
def allprep(file, numrows=None):
    df = pd.read_csv(file, dtype={'fullVisitorId': str, 'date': str}, nrows=numrows)
    df = date_conv(df)
    df = unpack(df)
    df = clean(df)
    return df

### Check

In [34]:
trainset = allprep('train.csv')
trainset['totals_transactionRevenue'].fillna(value=0, inplace=True)

test = allprep('test.csv')
test['totals_transactionRevenue'] = -99

In [35]:
trainset.head()

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,geoNetwork_city,geoNetwork_continent,geoNetwork_country,geoNetwork_metro,...,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,tS_adwordsCI_page,tS_adwordsCI_slot,tS_adwordsCI_gclId,tS_adwordsCI_adNetworkType,tS_adwordsCI_isVideoAd
0,Organic Search,2016-09-02,1131660440785968503,1472830385,1,1472830385,Izmir,Asia,Turkey,NaN,...,False,NaN,organic,NaN,google,NaN,NaN,NaN,NaN,True
1,Organic Search,2016-09-02,377306020877927890,1472880147,1,1472880147,NaN,Oceania,Australia,NaN,...,False,NaN,organic,NaN,google,NaN,NaN,NaN,NaN,True
2,Organic Search,2016-09-02,3895546263509774583,1472865386,1,1472865386,Madrid,Europe,Spain,NaN,...,False,NaN,organic,NaN,google,NaN,NaN,NaN,NaN,True
3,Organic Search,2016-09-02,4763447161404445595,1472881213,1,1472881213,NaN,Asia,Indonesia,NaN,...,False,google + online,organic,NaN,google,NaN,NaN,NaN,NaN,True
4,Organic Search,2016-09-02,27294437909732085,1472822600,2,1472822600,NaN,Europe,United Kingdom,NaN,...,True,NaN,organic,NaN,google,NaN,NaN,NaN,NaN,True


In [36]:
test.head()

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,geoNetwork_city,geoNetwork_continent,geoNetwork_country,geoNetwork_metro,...,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,tS_adwordsCI_page,tS_adwordsCI_slot,tS_adwordsCI_gclId,tS_adwordsCI_adNetworkType,tS_adwordsCI_isVideoAd,totals_transactionRevenue
0,Organic Search,2017-10-16,6167871330617112363,1508151024,2,1508151024,NaN,Asia,Singapore,NaN,...,NaN,organic,NaN,google,NaN,NaN,NaN,NaN,True,-99
1,Organic Search,2017-10-16,0643697640977915618,1508175522,1,1508175522,Zaragoza,Europe,Spain,NaN,...,NaN,organic,NaN,google,NaN,NaN,NaN,NaN,True,-99
2,Organic Search,2017-10-16,6059383810968229466,1508143220,1,1508143220,NaN,Europe,France,NaN,...,NaN,organic,NaN,google,NaN,NaN,NaN,NaN,True,-99
3,Organic Search,2017-10-16,2376720078563423631,1508193530,1,1508193530,Mountain View,Americas,United States,San Francisco-Oakland-San Jose CA,...,NaN,organic,NaN,google,NaN,NaN,NaN,NaN,True,-99
4,Organic Search,2017-10-16,2314544520795440038,1508217442,1,1508217442,San Jose,Americas,United States,San Francisco-Oakland-San Jose CA,...,NaN,organic,NaN,google,NaN,NaN,NaN,NaN,True,-99
